## Fune-tune the System -SGD Classifier

In [1]:
# setting

import sys
sys.path.append("..")

from module.utils import general_utils
from module.utils import data_prepare_utils
from module.utils import model_performance_utils
import module.utils.bin_class_utils as bin_class_utils

from sklearn.linear_model import SGDClassifier

from sklearn.pipeline import Pipeline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
target_attr = "label"
target_type = "binary"

estimator_name = "SGDClassifier"

sample_random_state = 24
target_encoding_random_state = 42
loss = 'log_loss'
model_random_state = 42
permutation_importance_random_state = 0

In [3]:
train_df = pd.read_csv('train_df_exp.csv')
train_cap_x_df, train_y_df = train_df.drop(columns=target_attr), train_df[[target_attr]]

balanced = train_y_df.shape[0] / (train_y_df[target_attr].nunique()*np.bincount(train_y_df[target_attr]))
balanced_and_normalized_dict = \
    dict(
        zip(
            train_y_df[target_attr].unique(),
            balanced/sum(balanced)
            )
        )

In [4]:
numerical_attr_list = []
categorical_attr_list = [attr for attr in train_cap_x_df.columns if attr not in numerical_attr_list and attr != target_attr]
attr_list = numerical_attr_list + categorical_attr_list

In [5]:
attrs_to_drop = ['app_score', 'his_on_shelf_time', 'task_id', 'spread_app_id', 'tags', 'dev_id', 'app_second_class', 'adv_prim_id', 'device_price']
preprocessor = bin_class_utils.build_preprocessing_pipeline(numerical_attr_list, categorical_attr_list, attrs_to_drop, target_type, target_encoding_random_state)
estimator = SGDClassifier(loss=loss, random_state=model_random_state, class_weight=balanced_and_normalized_dict)
dt_pipe = Pipeline([('preprocessor', preprocessor), ('estimator', estimator)])

#### 0. Tune Hyperparameters of Composite Estimator - Ranking Metrics

In [6]:
best_model = bin_class_utils.tune_hyperparameters_sgd(train_cap_x_df, train_y_df, dt_pipe)

Best estimator: {'estimator__alpha': 0.001, 'estimator__loss': 'log_loss', 'estimator__max_iter': 1000, 'estimator__penalty': 'l1', 'preprocessor__categorical__target_encoder__smooth': 'auto', 'preprocessor__numerical__imputer__strategy': 'mean'}
accuracy: 0.6943615751789977
ave_precision_score: 0.054408123414260476
roc_curve_auc: 0.6538739338832031


#### 1. Evaluate Tuned Composite Estimators

#### 2. Calibrate Composite Estimator If Required

#### 3. Check for False Discoveries

#### 4. Selection a Model

#### 5. Permutation Feature Importance

#### 6. Tune Classification Threshold for Classification - Classification Metrics

#### 7. Serialize Model and Classification Threshold

#### 8. Evaluate Model on the Test Set